# **Udacity: Intro to TensorFlow for Deep Learning**
## **Lesson 10 NLP: Recurrent Neural Networks**

This lessons extends on what was covered in lesson 9. It introduces recurrent neural networks, which are able to capture temporal dependences,that change over time.

This lesson would covers
- Different RNNs: Simple RNN, LSTMS, GRUs
- Text generation using NLP models.


##**Simple RNNS**

Simple recurrent neural networks, are networks which use outputs from previous time steps as additional inputs, alongside the current input.

for example   
- Inputs: $X_t + Y_{t-1}$
- Ouput: $Y_t$

I'm a bit effy around th calculation of the output.

But...

The general idea is that, previous output from the last time step are fed as additional inputs. The previous output is referred to as the state vector.

<br>

While simple RNN architecture are able to consider the past output when calculating it's new output, it is limited by how far back it can relate dependencies and for dependencies which occur over long period of time, a simple RNN would struggle to capture these dependencies.

## **Long term short term Memory**

LSTMs, were introduced to capture temporal dependencies which spam over longer periods of time. Unlike simple RNNS which have a single state vector, LSTMS have 2 state vectors, a Long term memory and a short term memory.

<br>

**Features of an LSTM**
- It's able to capture temporal dependencies spaming a long period of time
- It has 2 state vectors, long term and short term memory, which are used in calculating a new input
- LSTM feature gates: Forget, learn, remember and use gates.

<br>

**Workflow for an LSTM**
- At each time step, the LSTM has 2 state vectors: A long term memory and a short term memory
- The current time step input alongside the 2 state vectors are used to determine an output.
- The calculated output from the current time step, would be used as the short term memory for the next timestep
- The long term memory from the previous time step is updated:
  - Any pieces of the previous long term memory which is no longer relevant is removed, using the forget gate
  - Any new piece of relevant information is added to the long term memory, using the remember gate

<br>

**LSTM Gates**
- Forget gate: Determines which parts of the long term memory are no longer relevant and should be removed from memory.
- Learn gate: Learns new piece of informations using the current input and short term memory.

- Remember gate: Adds any relevant information that was learnt to the long term memory. The output of this gate is the new long term memory

- Use gate: This uses, the relevants parts of the long term memory and newly learnt information to calculate an output. The output is also used as the new short term memory for the next timestep.



##**Import Dependencies**

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

2.8.2


## **RNNs and LSTMs in code**

The LSTM layer is within `tf.keras.Layers.LSTM` [docs](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM). likewise with the RNN layer `tf.keras.Layers.SimpleRNN`.

<br>

Worth noting:
- We can directly pass, the output of an embedding layer to an LSTM layer without adding a flatten or GlobalAverage1D layer inbetween.

<br>

**Further resources:**   
The [recurrent neural network (RNN) with keras guide](https://www.tensorflow.org/guide/keras/rnn) is a really good supplementary introduction to RNNs.

Notes from the guide
- 3 built-in RNN layers: SimpleRNN, GRU, LSTM
- RNN can process input sequences in reverse.
- Feature recurrent dropout
- By default returns output at the last time step, but it can be configured to return a sequence instead for each time step.
- The layer can also be configured to return the final internal state vectors.
- Likewise we can also set the initial state of the RNN layer.

<br>

**Difference between layer and cell layer**
- "*the RNN cell process only a single timestep*"

### **Preparing text for SimpleRNN and LSTM layer**

Simplified look at using the SimpleRNN and LSTM layers. Both these layers would take inputs from the Embedding layer, which would return a vector in n dimensions for each token in the sequence.

To take a look at the input and given output, lets try it out on a simple dataset.

In [2]:
# sample text
# The great pretender - The Platters
lyrics = ["Oh-oh, yes, I'm the great pretender",
          "Pretending that I'm doing well",
          "My need is such I pretend too much",
          "I'm lonely, but no one can tell",
          "Oh-oh, yes, I'm the great pretender",
          "Adrift in a world of my own",
          "I've played the game but to my real shame",
          "You've left me to grieve all alone",
          "Too real is this feeling of make-believe",
          "Too real when I feel what my heart can't conceal",
          "Yes, I'm the great pretender",
          "Just laughin' and gay like a clown",
          "I seem to be what I'm not, you see",
          "I'm wearing my heart like a crown",
          "Pretending that you're still around"
          "Too real is this feeling of make-believe",
          "Too real when I feel what my heart can't conceal",
          "Yes, I'm the great pretender",
          "Just laughin' and gay like a clown",
          "I seem to be what I'm not, you see",
          "I'm wearing my heart like a crown",
          "Pretending that you're still around (still around)"]


In [3]:
# tokenize and pad the text

# deprecated in version 2.9.1
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# define a tokenizer and fit it to the text
The_great_pretender = Tokenizer(num_words=150, oov_token="<OOV>")
The_great_pretender.fit_on_texts(lyrics)


In [4]:
# display the word index
word_index = The_great_pretender.word_index
print(word_index)

{'<OOV>': 1, "i'm": 2, 'my': 3, 'the': 4, 'i': 5, 'a': 6, 'real': 7, 'oh': 8, 'yes': 9, 'great': 10, 'pretender': 11, 'too': 12, 'to': 13, 'what': 14, 'heart': 15, 'like': 16, 'pretending': 17, 'that': 18, 'is': 19, 'of': 20, 'still': 21, 'but': 22, 'this': 23, 'feeling': 24, 'make': 25, 'believe': 26, 'when': 27, 'feel': 28, "can't": 29, 'conceal': 30, 'just': 31, "laughin'": 32, 'and': 33, 'gay': 34, 'clown': 35, 'seem': 36, 'be': 37, 'not': 38, 'you': 39, 'see': 40, 'wearing': 41, 'crown': 42, "you're": 43, 'around': 44, 'doing': 45, 'well': 46, 'need': 47, 'such': 48, 'pretend': 49, 'much': 50, 'lonely': 51, 'no': 52, 'one': 53, 'can': 54, 'tell': 55, 'adrift': 56, 'in': 57, 'world': 58, 'own': 59, "i've": 60, 'played': 61, 'game': 62, 'shame': 63, "you've": 64, 'left': 65, 'me': 66, 'grieve': 67, 'all': 68, 'alone': 69, 'aroundtoo': 70}


In [5]:
# convert the lyrics to sequences
lyrics_sequence = The_great_pretender.texts_to_sequences(lyrics)

length_of_sequence = []
for sequence in lyrics_sequence:
  print(sequence)
  break

[8, 8, 9, 2, 4, 10, 11]


In [6]:
# display the average length of each sequence
length_of_sequence = []
for sequence in lyrics_sequence:
  length_of_sequence.append(len(sequence))


#src: https://www.geeksforgeeks.org/find-average-list-python/
def Average(lst):
    return sum(lst) / len(lst)
  

print(Average(length_of_sequence))

7.619047619047619


In [7]:
# Apply padding to the sequences
lyrics_sequence_padded = pad_sequences(lyrics_sequence, maxlen=7, padding='pre',
                                      truncating='post')


In [8]:
# define an embedding layer
Embedding = tf.keras.layers.Embedding(input_dim=150, output_dim=4, input_length=7)

In [9]:
# pass in a sequence from the lyric_sequence_padded to the embedding layer
output = Embedding(np.array(sequence))
print(output)

tf.Tensor(
[[-0.03605381 -0.04465077 -0.03381597 -0.03534323]
 [ 0.01556465 -0.03940675  0.01276333  0.00853235]
 [ 0.00911007 -0.04884337  0.01232371 -0.02719672]
 [-0.0399784  -0.02804041  0.02798759 -0.04888147]
 [-0.02194605 -0.00159215  0.04856716 -0.01162859]
 [-0.0399784  -0.02804041  0.02798759 -0.04888147]
 [-0.02194605 -0.00159215  0.04856716 -0.01162859]], shape=(7, 4), dtype=float32)


we now have a vector of 4 dimensions representing of each tokens in the sequence.

A thought on the embedding layer
- i don't think the embedding layer is trainable, provided it's function is to simply convert the tokens into a vector representation, i can't really see how any optimization is needed.

Unless the scale or way in which the conversion is done needs to be different to get better results, i'm not sure.

Looking into the above text, *i don't think the embedding layer is trainable*

<br>

**Is the embedding layer trainable?**

The short answer is that, the embedding layer is trainable, it is not enough to just map each token to vector.
<br>

- At it's core, the embedding layer maps positive integers (tokens) into dense vectors of fixed size (word embeddings).
- ideally we would like words with similar semantic context or sentiment, to have similar vector representations, so words like man and woman should have fairly similar vectors. Likewise words like apple, orange, banana, pears should be within the same general cluster, in our n dimensional space for our word embedding.
- We would not be able to achieve this by randomly mapping our tokens into vectors. Hence our vector representation would need to be optimzied to group similar words to be within the same latent space in our word embedding.

<br>

**Resources**
- https://stats.stackexchange.com/questions/324992/how-the-embedding-layer-is-trained-in-keras-embedding-layer
- https://www.youtube.com/watch?v=5MaWmXwxFNQ

<br>

**More questions**
- What is word2Vec, skip-gram?

### **Understanding the keras LSTM layer**

In [10]:
# pass the output of the embedding layer to the LSTM layer
LSTM = tf.keras.layers.LSTM(units=4)

embedding_output = Embedding(np.array([sequence]))
LSTM_output = LSTM(embedding_output)

print("Input: {}".format(np.array([sequence])))
print("\nEmbedding layer output: ")
print(embedding_output)
print("\nLSTM layer output: ")
print(LSTM_output)

Input: [[17 18 43 21 44 21 44]]

Embedding layer output: 
tf.Tensor(
[[[-0.03605381 -0.04465077 -0.03381597 -0.03534323]
  [ 0.01556465 -0.03940675  0.01276333  0.00853235]
  [ 0.00911007 -0.04884337  0.01232371 -0.02719672]
  [-0.0399784  -0.02804041  0.02798759 -0.04888147]
  [-0.02194605 -0.00159215  0.04856716 -0.01162859]
  [-0.0399784  -0.02804041  0.02798759 -0.04888147]
  [-0.02194605 -0.00159215  0.04856716 -0.01162859]]], shape=(1, 7, 4), dtype=float32)

LSTM layer output: 
tf.Tensor([[-0.00894124  0.02133395  0.00866948  0.00472196]], shape=(1, 4), dtype=float32)


So what has happened??
- it looks like the number of units correspond to the shape of the output, so if there is 1 unit it would produce a shape of (1, 1) and if there are 4 units it would produce a shape of (1, 4).
- recap output of embedding layer is vector of n dimension representation of a sequence.

<br/>

Still doesn't answer what exactly its doing??
- What does the output of the LSTM layer mean??

<br/>

what happens if we ask it to 
- return sequences
- return state
- go backwards

**Set return_sequences to True**

In [11]:
# lstm layer with return sequence = True
LSTM_return_sequence = tf.keras.layers.LSTM(units=4, return_sequences=True)
LSTM_return_sequence_output = LSTM_return_sequence(embedding_output)

print("Input: {}".format(np.array([sequence])))
print("\nLSTM layer output: ")
print(LSTM_return_sequence_output)

Input: [[17 18 43 21 44 21 44]]

LSTM layer output: 
tf.Tensor(
[[[-0.00890752  0.00129125 -0.00724328 -0.00319293]
  [-0.00538249  0.00093071 -0.00432462 -0.00776296]
  [-0.00702416 -0.00165671 -0.00709613 -0.01226564]
  [-0.00721529 -0.00339077 -0.01583953 -0.01056521]
  [ 0.00088901 -0.00384751 -0.01626903 -0.00681015]
  [-0.00045053 -0.00519655 -0.02308342 -0.00636325]
  [ 0.0067431  -0.005319   -0.02191803 -0.00356569]]], shape=(1, 7, 4), dtype=float32)


with return sequence set to True as it iterates through the sequence, it would return a value for each timestep, as there are 7 tokens in a sequence, it returns 7 values and since we have 4 units in our LSTM for each time step it returned 4 value.

still don't fully understand what the output mean

**Understanding the output of an LSTM layer**

Notes from machine learning mastery.
- The LSTM is a class of recurrent neural networks which contains internal gates (Learn, forget, remember and use gates).
- This class of recurrent neural networks are designed to resolve the vanishing gradient problem, as it is able to capture longer temporal dependencies.
- An LSTM layer can be defined with n number of LSTMs cell. each cell contains,
  - an internal cell state, *c*
  - outputs a hidden state *h*

So the output of an LSTM is the hidden state *h*.

<br>

By setting 
- **return_sequence to True**, we are able to access the hidden state at each time step, in our sequence.
- **return_state to True**, this would return the final internal cell states of each cell in the LSTM layer. So it would return *c* and *h*.

We would typically set the `return_state to True`, when we want to initialize the states of another LSTM layer with the same number of cells.

<br>

Cool...

But this has still not answered the question of what the internal cell state and hidden cell states are.




<br>

**Resources**
- https://stackoverflow.com/questions/67970519/what-does-tensorflow-lstm-return

- https://machinelearningmastery.com/return-sequences-and-return-states-for-lstms-in-keras/
 

In [12]:
# Example used in stackoverflow question
# Set return_sequence and return_states to True

# input shape: Batch size, length of sequence, embedding dimension
tensor = tf.random.normal(shape=[2, 2, 2])
lstm = tf.keras.layers.LSTM(units=4, return_sequences=True, return_state=True)
hidden_state_at_each_time_step, final_hidden_state, final_cell_state = lstm(tensor)

print("Input: {}".format(tensor))
print("\nHidden_state_at_each_time_step:\n", hidden_state_at_each_time_step)
print("\nFinal hidden state: \n", final_hidden_state)
print("\nFinal cell state: \n", final_cell_state)

Input: [[[-1.4442931  -0.10136034]
  [ 1.1435893   0.13428247]]

 [[-0.10974066  0.32672334]
  [-0.62598217  0.93750346]]]

Hidden_state_at_each_time_step:
 tf.Tensor(
[[[ 0.21792501  0.14318192 -0.15826057  0.0583761 ]
  [ 0.01737139 -0.01054842  0.04750073 -0.04627062]]

 [[ 0.04343653 -0.02368674  0.00332432  0.01399184]
  [ 0.20665176 -0.06394225 -0.01999095  0.07628803]]], shape=(2, 2, 4), dtype=float32)

Final hidden state: 
 tf.Tensor(
[[ 0.01737139 -0.01054842  0.04750073 -0.04627062]
 [ 0.20665176 -0.06394225 -0.01999095  0.07628803]], shape=(2, 4), dtype=float32)

Final cell state: 
 tf.Tensor(
[[ 0.04193182 -0.02323447  0.10621233 -0.08096145]
 [ 0.41175985 -0.1119123  -0.05270814  0.1616236 ]], shape=(2, 4), dtype=float32)


**WTF...**

**Breakdown of the output**


For an input of shape `[2, 2, 2]`, we have 2 batches of 2 sentences.
``` python
[[[-0.4942633   1.0938902 ], [-0.27461976 -0.2292373 ]]
 [[-0.34839615 -0.39708054], [ 0.21653225  2.0741708 ]]]
```
Where `[-0.4942633   1.0938902 ]` for example is our word embedding for a single token.

So an example input for the above word embedding is
``` python
example sequence of tokens = [[10, 11], [12, 13]]

# 10 -> [-0.4942633   1.0938902 ]
# 11 -> [-0.27461976 -0.2292373 ]
# 12 -> [-0.34839615 -0.39708054]
# 13 -> [ 0.21653225  2.0741708 ]

# decoded even futher as another example 
["hello, you"], ["Good food"]

# hell0 -> 10
# you -> 11
# Good -> 12
# food -> 13
```

With return sequence and state set to True. We have
- The hidden state $h_{t}$ returned at each time step.
- The final hidden cell state $h$ returned
- The final cell state $c$ returned.


Looking carefully at the output, you should see that the final hidden cell state is just the last hidden cell state for each sentence.

**A quick note on the dimensions.**

For  a given input dimension [X, Y, Z]. X would be the batch dimension and determines the number of initial element in the array. So if
- X = 2 we would have an array containing 2 elements. $[1, 2]$
- X = 5, we would have 5 elements in the array. $[1, 2, 3, 4, 5]$

The next dimension Y would determine the number of element within the initial set of elements. so if our dimensions are
- [1, 2, Z], we would have an array structured like this $[ [[], []] ]$
Admittedly it looks confusing, but the array would contain a single element, which in turn contains 2 elements.

stacking that further, the 3rd dimension determines the number of elements within the last set. For example an array with a dimension of

- [1, 2, 3] could like this, [[[1, 2, 3], [4, 5, 6]]]
- [3, 4, 6] could look like this 

```
[[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24]],
[[11, 12, 13, 14, 15, 16], [17, 18, 19, 20, 21, 22], [23, 24, 25, 26, 27, 28], [29, 30, 31, 32, 33, 24]],
[[21, 22, 23, 24, 25, 26], [27, 28, 29, 30, 31, 32], [33, 34, 35, 36, 37, 38], [39, 40, 41, 42, 43, 44]]]
```



In [13]:
# lets try passing in the example sequence above to an LSTM layer and view what the output is

# single batch with sequence length of 4 and 6 embedding dimensions.
test_sequence = np.array([[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12], [13, 14, 15, 16, 17, 18], [19, 20, 21, 22, 23, 24]]], dtype=np.float32)
LSTM_test_2 = tf.keras.layers.LSTM(2)

test_sequence_output = LSTM_test_2(test_sequence)

print("Results: {}".format(test_sequence_output))


Results: [[ 3.7682990e-18 -7.6085186e-01]]


In [14]:
# set go_backwards to True
LSTM_test_3 = tf.keras.layers.LSTM(2, go_backwards=True)

test_sequence_output_1 = LSTM_test_3(test_sequence)

print("Results: {}".format(test_sequence_output_1))


Results: [[ 0.08352461 -0.00242911]]


reading the docs, it would process the sequence backwards and returns the reversed sequence.

In [15]:
# set return sequence to True

LSTM_test_4 = tf.keras.layers.LSTM(2, return_sequences=True)

test_sequence_output_2 = LSTM_test_4(test_sequence)

print("Results: {}".format(test_sequence_output_2))


Results: [[[-1.6088609e-02  1.3590582e-01]
  [-9.9099570e-06  1.4932834e-02]
  [-5.1123448e-09  1.3205848e-03]
  [-2.5271356e-12  1.0443183e-04]]]


Looking at the output,
```
[[[ 4.5328138e-06 -1.6741604e-03] --> Output for the 2 units on the first word embedding in the batch.
  [ 2.3820494e-09 -1.7256550e-03] --> Output for the 2 units on the second word embedding in the batch.
  [ 1.2384534e-12 -1.8277960e-03] --> Output for the 2 units on the third word embedding in the batch.
  [ 6.4388152e-16 -1.9796446e-03] --> Output for the 2 units on the fourth sequence in the batch.]]
```


In [16]:
# set return state to True
LSTM_test_5 = tf.keras.layers.LSTM(2, return_state=True)

test_sequence_output_5 = LSTM_test_5(test_sequence)

print("Results: {}".format(test_sequence_output_5))


Results: [<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.7615941 , -0.00511423]], dtype=float32)>, <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.7615941 , -0.00511423]], dtype=float32)>, <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[-0.9999999, -3.9807653]], dtype=float32)>]


### **Understanding the SimpleRNN Layer**

**Quick recap**   
SimpleRNN are the basic implementation of recurrent neural networks (A class of neural networks which is able to handle sequential data such as text, time series, speech, etc..)

<br>

At each time step, the SimpleRNN output a prediction and a state vector. The state vector from previous time steps is then used as an additional input in calculating the next prediction at the next time step.

<br>

SimpleRNN are limited in it's capacity to capture dependencies spaning long periods of time.


Resources   
- https://machinelearningmastery.com/an-introduction-to-recurrent-neural-networks-and-the-math-that-powers-them/
- https://machinelearningmastery.com/understanding-simple-recurrent-neural-networks-in-keras/


Looking at the docs for simpleRNN there are loads of different option that we can play with. For now i think the basic input argument we need to care about are

- units: Number of SimpleRNN in the layer

- activation: Activation function used,

- use_bias: (Boolean), decides if layer should use a bias vector.

- dropout: (float) decides number of units to drop during forward pass

- recurrent_dropout: (float) decides number of units to drop during forward pass of recurrent state???

- return_sequences: (Boolean), returns the last output or return full sequence

- return_state: (Boolean), returns last state

- go_backwards: (Boolean), process input_sequence backwards and returns reversed sequence

- stateful: (Boolean), return last state for each sample in the batch


In [18]:
# play around with a SimpleRNN

# input sequence
sample_sequence = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])

# define an embedding layer
embedding_layer = tf.keras.layers.Embedding(input_dim=100, output_dim=8, input_length=4)

embedding_output = embedding_layer(sample_sequence)

print(f"Input_sequence: {sample_sequence}")
print(f"embedding_output: {embedding_output}")


Input_sequence: [[1 2 3 4]
 [5 6 7 8]]
embedding_output: [[[ 0.0071786  -0.02770999 -0.01056663  0.03591186  0.03259296
   -0.0434543   0.00465118 -0.04895863]
  [ 0.04632975 -0.00595964 -0.03106047 -0.00852524 -0.0071097
    0.02627807  0.03068611 -0.04760075]
  [ 0.01149998  0.04637103 -0.04353411  0.03820454 -0.0167288
    0.04148007  0.02381236 -0.01381072]
  [-0.00913991  0.02020909 -0.02490957  0.017342   -0.01278111
    0.01980628 -0.01485886  0.02753275]]

 [[ 0.01001146  0.01278545  0.03155548 -0.04833524 -0.02118723
   -0.03614746  0.00069385  0.0216359 ]
  [ 0.03733946  0.04492016  0.01998489 -0.00531695  0.0302333
   -0.02009546 -0.00944006  0.02976966]
  [-0.02754343 -0.01465994  0.0043788   0.00353374  0.02820127
    0.04190229  0.01130612  0.00886506]
  [-0.00214953 -0.02354258 -0.02432892 -0.02066363 -0.01827277
    0.03656251  0.03536358 -0.04086025]]]


In [19]:
# basic SimpleRNN
simpleRNN_layer_0 = tf.keras.layers.SimpleRNN(units=5)
simpleRNN_layer_0_output = simpleRNN_layer_0(embedding_output)

print(f"Output of simpleRNN: {simpleRNN_layer_0_output}")

Output of simpleRNN: [[-0.0035262   0.00762389 -0.00519237 -0.02459231  0.05055945]
 [-0.03090524  0.03047003 -0.0064576   0.11406601  0.08437794]]


output from the 5 simpleRNN in the layer, for the 2 samples in the batch.

In [20]:
# SimpleRNN with return sequence set to True

simpleRNN_layer_1 = tf.keras.layers.SimpleRNN(units=7, return_sequences=True)
simpleRNN_layer_1_output = simpleRNN_layer_1(embedding_output)

print(f"Output of simpleRNN: f{simpleRNN_layer_1_output}")


Output of simpleRNN: f[[[ 4.67170514e-02  2.57526711e-02  2.19654310e-02 -7.40337968e-02
   -3.85756120e-02 -1.63747054e-02 -2.88166408e-03]
  [-3.39741930e-02  7.66357686e-03 -3.74802686e-02 -4.72790487e-02
    4.18617809e-03  1.86630897e-02 -7.19470456e-02]
  [-3.36450413e-02  8.45279172e-02  7.30351405e-03  6.56275600e-02
   -3.30622005e-03 -2.46235561e-02  1.13326246e-02]
  [ 4.19856124e-02  6.94798827e-02  5.03926985e-02 -2.04432793e-02
    5.17919920e-02 -5.59812237e-04 -5.30481860e-02]]

 [[-7.80791370e-03 -5.34884445e-02 -4.55878153e-02  4.10600640e-02
   -5.71024092e-03  3.20048258e-02  1.91510711e-02]
  [ 3.83636951e-02 -3.65029946e-02 -4.65968624e-05  3.88131700e-02
   -7.57143274e-02  2.90241241e-02  8.19245279e-02]
  [ 6.81796297e-02  1.02709548e-03 -1.34778265e-02 -8.17172416e-03
   -3.13379727e-02  1.39801670e-02  3.46749835e-02]
  [ 3.24992128e-02 -4.28902637e-03 -4.15240116e-02 -3.92574398e-03
   -5.29155368e-03 -3.94932553e-02 -3.66065167e-02]]]


output from a simpleRNN layer containing 7 units. For each batch and each sample in the batch it produces 7 output. Hence final shape is [2, 4, 7]

In [21]:
# SimpleRNN with return state set to True

simpleRNN_layer_2 = tf.keras.layers.SimpleRNN(units=11, return_state=True)
simpleRNN_layer_2_output, final_state = simpleRNN_layer_2(embedding_output)

print(f"output of simpleRNN with return state set to True")
print(f"{simpleRNN_layer_2_output}")
print(f"{final_state}")

output of simpleRNN with return state set to True
f[[-0.00899099 -0.08651289 -0.06966928 -0.06566335 -0.02472426  0.02238877
  -0.05144454 -0.02784575  0.08847877 -0.0409691  -0.03349891]
 [ 0.00236626  0.13123365  0.05066454  0.00946779  0.06558042 -0.04847437
   0.0884525   0.0780066  -0.05067851  0.07823572 -0.02735425]]
f[[-0.00899099 -0.08651289 -0.06966928 -0.06566335 -0.02472426  0.02238877
  -0.05144454 -0.02784575  0.08847877 -0.0409691  -0.03349891]
 [ 0.00236626  0.13123365  0.05066454  0.00946779  0.06558042 -0.04847437
   0.0884525   0.0780066  -0.05067851  0.07823572 -0.02735425]]


it looks like the final states are the same as the outputs. 😒

I wonder if my understanding of the state vector is wrong?? Could it be that the state vector or just the states are the same as the output of the SimpleRNN??

In [24]:
# SimpleRNN with return states and return sequence set to True

simpleRNN_layer_3 = tf.keras.layers.SimpleRNN(units=5, return_sequences=True, return_state=True)
simpleRNN_layer_3_output, final_state = simpleRNN_layer_3(embedding_output)

print(f"Output of simpleRNN with return sequences and states set to True")
print(f"{simpleRNN_layer_3_output}")
print(f"\n{final_state}")

Output of simpleRNN with return sequences and states set to True
[[[ 0.03062244 -0.02164007 -0.01669262  0.02174721  0.0110234 ]
  [-0.06658126 -0.0384743  -0.00944478  0.06394269 -0.03552142]
  [-0.0083191  -0.01397501  0.05373615 -0.02540039  0.0011354 ]
  [-0.03204141  0.08703158  0.0084386  -0.00434022 -0.00166453]]

 [[ 0.0048805   0.00900241 -0.00591772 -0.05879032 -0.04276478]
  [ 0.0667311   0.04170684 -0.07011628 -0.09220314 -0.06993692]
  [ 0.07790264  0.00506384 -0.07255677  0.04522913 -0.00602574]
  [-0.01184071 -0.08007074  0.00045335  0.12782802 -0.00818433]]]

[[-0.03204141  0.08703158  0.0084386  -0.00434022 -0.00166453]
 [-0.01184071 -0.08007074  0.00045335  0.12782802 -0.00818433]]


yeah it looks like the output and states are the same.

Question why are states not passed between batches?
- Looking at the output it seems that the outputs are the same as the states. For a batch containing multiple samples which we assume as independent from each other it would not make any sense to use the last prediction from the previous sample as the input state.

<br>

Just like with the LSTM, the final output and the hidden states are the same, I'm still not entirely sure about the final cell state in an LSTM layer.

### **Understanding GRU Layers**

##**Using Conv1D Layers**
